# Part 2: Baseline Model using Pre-Trained Language Model

This python notebook corresponds directly to the section 4.4 in the final thesis report. 

### Mount Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

### Load Required Libraries

In [ ]:
pip install transformers

In [ ]:
import os
import sys
import json
import random
import time
import re
import datetime
import pickle

from tqdm import tqdm

import numpy as np

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix


from transformers import RobertaForSequenceClassification, BertForSequenceClassification,  DistilBertForSequenceClassification 
from transformers import BertTokenizer, DistilBertTokenizer, RobertaTokenizer
from transformers import BertConfig, DistilBertConfig, RobertaConfig
from transformers import AdamW, get_linear_schedule_with_warmup 

from transformers import logging
logging.set_verbosity_error()

import warnings
warnings.filterwarnings('ignore')

### Model Configs

In [ ]:
# set path variables
basepath = '/content/gdrive/MyDrive/ljmu-ms-thesis/'
datapath = '/content/gdrive/MyDrive/ljmu-ms-thesis/data/'
modelpath =  '/content/gdrive/MyDrive/ljmu-ms-thesis/model/'

In [ ]:
# common config settings
dataset = 'amazon_data.json'
dataset_path = os.path.join(datapath, dataset)

EPOCHS = 10
TOTAL_ITERATIONS = 3
TEST_SIZE = 0.2
MAX_LEN = 32
BATCH_SIZE = 16
RANDOM_STATE = 2022
LEARNING_RATE = 2e-5
EPS = 1e-8
SEED_VAL = 42

In [ ]:
# choose one model at a time
#model_name = 'bert'
#model_name = 'roberta'
model_name = 'distilbert'

### Load Data

In [ ]:
def load_dataset(filename):
    dataset = []
    with open(filename) as f:
        for line in f:
            entry = {}
            
            line = line.strip()
            d = json.loads(line)
            
            entry['sentence'] = d['sentence']
            entry['label'] = int(d['label'])
            dataset.append(entry)
    return dataset

In [ ]:
dataset = load_dataset(dataset_path)

### Data Pre-processing

In [ ]:
# Initialise tokenizer
def get_tokenizer(model_name):
    if model_name == 'distilbert':
        tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)
    elif model_name == 'bert':
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    elif model_name == 'roberta':
        tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
    return tokenizer

tokenizer = get_tokenizer(model_name)

In [ ]:
# Tokenization and sentence embedding
def encode_all(sentences, model_name):  
    input_ids = []
    for data in sentences:
        input_ids.append(tokenizer.encode(data, add_special_tokens=True))
    return input_ids

In [ ]:
# adding attention mask
def get_attn(input_ids):
    attention_masks = []
    for sent in input_ids:
        att_mask = [int(token_id > 0) for token_id in sent]
        attention_masks.append(att_mask)
    return attention_masks

In [ ]:
# Create Dataset
def create_dataset(dataset):
    all_data = []
    for data in tqdm(dataset):
        input_ids = []
        input_ids.append(tokenizer.encode(data['sentence'].lower()))
        input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', value=0, truncating='post', padding='post')
        input_ids = torch.tensor(input_ids)
        
        attn_mask = torch.tensor(get_attn(input_ids))

        entry = {}
        entry['raw_sentence'] = data['sentence']
        entry['sentence'] = input_ids[0]
        entry['sentence_mask'] = attn_mask[0]
        
        all_data.append((entry, data['label']))
    return all_data

In [ ]:
dataset_list = create_dataset(dataset)

### Training & Validation Split

In [ ]:
trainset, validationset = train_test_split(dataset_list, random_state=RANDOM_STATE, test_size=TEST_SIZE)

In [ ]:
def save_dataset():
    with open(modelpath + 'trainset-' + model_name +'.data', 'wb') as f:
        pickle.dump(trainset, f)

    with open(modelpath + 'validationset-' + model_name + '.data', 'wb') as f:
        pickle.dump(validationset, f)

In [ ]:
save_dataset()

### Data Loaders

In [ ]:
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(validationset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Model Initialisation

In [ ]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

def reset_model(model_name):
    if model_name == 'distilbert':
        model = DistilBertForSequenceClassification.from_pretrained(
                                                'distilbert-base-uncased',
                                                num_labels = 2, 
                                                output_attentions = True, 
                                                output_hidden_states = True, 
                                            )
    elif model_name == 'roberta':
        model = RobertaForSequenceClassification.from_pretrained(
                                                'roberta-base',
                                                num_labels = 2, 
                                                output_attentions = True, 
                                                output_hidden_states = True, 
                                            )
    elif model_name == 'bert':
        model = BertForSequenceClassification.from_pretrained(
                                                'bert-base-uncased',
                                                num_labels = 2, 
                                                output_attentions = True, 
                                                output_hidden_states = True, 
                                            )
    model.to(device)
    
    optimizer = AdamW(model.parameters(),
                  lr = LEARNING_RATE,
                  eps = EPS 
                )

    total_steps = len(train_loader) * EPOCHS

    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    return model, optimizer, scheduler

### Utility

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

### Training the Model

In [ ]:
random.seed(SEED_VAL)
np.random.seed(SEED_VAL)
torch.manual_seed(SEED_VAL)
torch.cuda.manual_seed_all(SEED_VAL)

In [ ]:
def train(model, optimizer, scheduler):
    total_loss = 0
    model.train()
    for step, (batch, labels) in enumerate(train_loader):
        b_input_ids = batch['sentence'].to(device)
        b_input_mask = batch['sentence_mask'].to(device)
        b_labels = labels.to(device)
        model.zero_grad()        
        
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
                
        loss = outputs[0]

        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_loader)

### Testing the Model

In [ ]:
def test(model):
    
    t0 = time.time()
    model.eval()
    
    all_predictions = []
    all_labels = []
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch, b_labels in test_loader:
        
        with torch.no_grad():        
            outputs = model(batch['sentence'].to(device), 
                            attention_mask=batch['sentence_mask'].to(device))
        
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        prediction = list(np.argmax(logits, axis=1).flatten())
        all_predictions.extend(prediction)
        all_labels.extend(label_ids.flatten())
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    
    matrix = confusion_matrix(all_predictions, all_labels)

    tp = matrix[0][0]
    fp = matrix[0][1]
    fn = matrix[1][0]
    tn = matrix[1][1]

    accuracy = eval_accuracy/nb_eval_steps
    f1 = f1_score(all_predictions, all_labels, average = 'macro')
    precision = precision_score(all_predictions, all_labels, average = 'macro')
    recall = recall_score(all_predictions, all_labels, average = 'macro')
    
    return [f1, precision, recall, accuracy]

In [ ]:
def save_model(filename = modelpath + model_name +'_baseline.pb'):
    torch.save(model, filename)

### Run the model

In [ ]:
# Run the model
def run(model, optimizer, scheduler):
    loss_values = []
    best_results = [0, 0, 0, 0]
    t0 = time.time()
    
    for epoch_i in range(0, EPOCHS):
        train(model, optimizer, scheduler)
        elapsed = format_time(time.time() - t0)
        print('Epoch {:} / {:}    Elapsed: {:}.'.format(epoch_i + 1, EPOCHS, elapsed))
        results = test(model)
        if best_results[0] < results[0]:
            best_results = results
            save_model()
 
    return best_results

In [ ]:
# Display the results
all_results = []
t0 = time.time()

for iteration in range(TOTAL_ITERATIONS):
    it0 = time.time()
    print('-'*50)
    print('Iteration {:2d}'.format(iteration+1))
    print('-'*50)
    model, optimizer, scheduler = reset_model(model_name)
    result = run(model, optimizer, scheduler)
    all_results.append(result)
    print('-'*50)
    print('Result for this iteration: ', result)
    print('Time taken for this iteration: {:}'.format(format_time(time.time() - it0)))

# Final results is the average of all the iterations
final_results = [sum(value)/len(value) for value in zip(*all_results)]

print('-'*50)
print('Final Results for the Baseline ' + upper(model_name) + ' model')
print('-'*50)
print('F1-score: {0:.4f}'.format(final_results[0]))
print('Precision: {0:.4f}'.format(final_results[1]))
print('Recall: {0:.4f}'.format(final_results[2]))
print('Accuracy: {0:.4f}'.format(final_results[3]))
print('Time Taken: {:}'.format(format_time(time.time() - t0)))
print('-'*50)
